In [ ]:
!pip install tensorflow pandas matplotlib scikit-learn --quiet

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = '/content/drive/My Drive/DadosTcc2'

try:
    for item in os.listdir(folder_path):
        print(item)
except FileNotFoundError:
    print(f"Error: The folder was not found")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
labels_path = os.path.join(folder_path, 'labels2.csv')
labels_df = pd.read_csv(labels_path)

labels_df['image_path'] = labels_df['image'].apply(lambda x: os.path.join(folder_path, 'dataset', x + '.jpg'))

labels_df.head()

In [ ]:
train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42)

print("Treino:", len(train_df))
print("Validação:", len(val_df))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=False
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col=['contrast','alignment','proximity'],
    target_size=(224,224),
    batch_size=8,
    class_mode='raw'
)


val_gen = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col=['contrast','alignment','proximity'],
    target_size=(224,224),
    batch_size=8,
    class_mode='raw',
    shuffle=False
)

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(3, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=[
                tf.keras.metrics.BinaryAccuracy(name='bin_acc'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                'accuracy']),

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    verbose=1
)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(history.history['bin_acc'], label='Treino')
plt.plot(history.history['val_bin_acc'], label='Validação')
plt.title('Evolução da acurácia')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(history.history['precision'], label='Treino')
plt.plot(history.history['val_precision'], label='Validação')
plt.title('Evolução da Precisão')
plt.xlabel('Época')
plt.ylabel('Precisão')
plt.legend()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(history.history['recall'], label='Treino')
plt.plot(history.history['val_recall'], label='Validação')
plt.title('Evolução do Recall')
plt.xlabel('Época')
plt.ylabel('Recall')
plt.legend()
plt.show()